<h1 align = "center">Επεξεργασία Φωνής και Φυσικής Γλώσσας</h1>
<h2 align = "center">3ο Εργαστήριο (Προπαρασκευή) </h2>
<h3 align = "center">Sentiment Analysis </h3>
<h3 align = "center"> Θεoδωρόπουλος Νικήτας -03115185</h3>
<h3 align = "center"> Καλλιώρα Δωροθέα - 03115176</h3>


### Εισαγωγή
Στην παρούσα εργαστηριακή άσκηση υλοποιούμε ένα μοντέλο για την επεξεργασία και την κατηγοριοποίηση κειμένων με χρήση Deep Neural Networks (**DNNs**), και την βιβλιοθήκη **PyTorch**. Για κάθε δείγμα εισόδου σε μορφή κειμένου δημιουργούμε διανυσματικές αναπαραστάσεις με χρήση pretrained word embeddings λέξεων. Συγκεκριμένα χρησιμοποιήσαμε [**GloVe**](https://nlp.stanford.edu/projects/glove/) αναπαραστάσεις $(50d,6B)$

Στόχος είναι η εκπαίδευση μοντέλων για ανάλυση συναισθήματος (**sentiment analysis**) σε προτάσεις. Θα δουλέψουμε πάνω στα εξής δύο corpora:

* **Sentence Polarity Dataset** [Pang and Lee, 2005](http://www.cs.cornell.edu/people/pabo/movie-review-data/). To dataset   
  περιέχει 5331 θετικές
  και 5331 αρνητικές κριτικές ταινιών, από το Rotten Tomatoes και   είναι binary-classification πρόβλημα (positive, negative).
  
  
  
* **Semeval 2017 Task4-A** [Rosenthal et al., 2017](http://alt.qcri.org/semeval2017/task4/index.php?id=data-and-tools). To dataset       αυτό περιέχει tweets τα οποία
  είναι κατηγοριοποιημένα σε 3 κλάσεις (positive, negative,         neutral) με 49570 παραδείγματα
  εκπαίδευσης και 12284 παραδείγματα αξιολόγησης.
  

### Κυρίως Μέρος

#### 1: Προεπεξεργασία των δεδομένων: 
Αρχικά έγινε κατάλληλη προεπεξεργασία των δεδομένων με χρήση των κλάσεων Dataset, Dataloader. Τα labels αντιστοιχίζονται σε αριθμούς και στα κείμενα γίνεται κατάλληλο tokenization και αντιστοίχιση των λέξεων σε embeddings indexes. Χρησιμοποιηθήκε ο **NLTK** sentence tokenizer. 

Για να μπορέσει να γίνει επεξεργασία των δεδομένων θα πρέπει τα δέιγματα να έχουν το ίδιο μέγεθος για τον λόγο αυτό επιλέχθηκε το σταθερό μέγεθος **60** λέξεων, και έγινε κατάλληλο padding ή μείωση. Το πραγματικό μήκος αποθηκεύεται. Τα δεδομένα μετατρέπονται σε Tensors απο την κλάση Dataloader


#### 2: Μοντέλο 

Σχεδιάζουμε την αρχιτεκτονική του Νευρωνικού Δικτύου. Με χρήση ενός **embedding layer** δημιουργούμε μια συνεχή διανυσματική αναπαράσταση για κάθε όρο της πρότασης, και για το δυνολικό δείγμα λαμβάνοντας τον μέσο όρο. Ο μέσος όρος προκύπτει διαιρώντας το άθροισμα με το πραγματικό μήκος του κειμένου, (τα embeddings των padded elements αντιστοιχίζονται στο μηδενικό διάνυσμα).

Το embedding layer όπως έχουμε δει αντιστοιχίζει κοντα λέξεις που είναι σημασιολογικά κοντά. Τα embeddings αρχικοποιούνται με τα pretrained GloVe embeddings. Και έπειτα παγώνουν $(requires\_grad=False)$.

Τα διανύσματα προβάλονται απο το embedding layer μέσω μιας μη γραμμικής συνάρτησης ενεργοποίησης **(ReLU)** σε έναν ενδιάμεσο χώρο 100 διαστάσεων. Το τελευταίο layer προβάλει τα διανύσματα στον χώρο εξόδου. Για Binary Classification επιλέχθηκε output layer μεγέθους **1**, για compatibillity με το BCELoss. Για την κατηγοριοποίηση τριών κλάσεων έγινε αντιστοίχιση $\mathbb{R}^{100} \rightarrow \mathbb{R}^3$.

Υλοποιούμε το Forward pass εφαρμόζοτας σε κάθε mini-batch τους παραπάνω μετασχηματισμούς. 


#### 3: Εκπαίδευση

Τα παραδέιγματα οργανώνονται σε mini-batches μεγέθους 128. Μεγαλύτερη τιμή είναι προτιμότερη για το *Semeval2017A* dataset λόγω του μεγάλου αριθμού δεδομένων. Εκτελούμε σε κάθε βήμα Stochasitc Gradient Descend. 

Για την εκπαίδευση χρησιμοποιούμε τα εξής:

* Κριτήριο: **BCEWithLogitsLoss** για δεδομένα 2 κλάσεων. **CrossEntropyLoss** για δεδομένα 3 κλάσεων. 


* Παράμετροι: Το learning rate επιλέχθηκε $lr = 0.0001$, διότι παρατηρήσαμε οτι μεγαλύτερες τιμές οδηγούν σε μεγάλωσες ταλαντωσεις του loss και μειώνουν την απόδοση. Οι παράμετροι των μοντέλων με $requires\_grad=True$ βελτιστοποιούνται με Gradient Descend


* Optimizer: Επιλέγουμε **Adam** καθώς χρησιμοποιείται ευρέως στην βιβλιογραφία και έχει γενικά καλα αποτελέσματα. Επίσης προσαρμόζει αυτόματα την ταχύτητα ενημέρωσης των μαρών. 


Αξιολογούμε το μοντέλο στο τέλος κάθε εποχής, εκτυπώνοντας το train loss. Παρακάτω για τα δύο Data sets εκπαιδεύουμε τα μοντέλα και παρουσιάζουμε τα τελικά αποτελέσματα: 
* **Classification Report** (F_1_score (macro), recall (macro))
* Διαγράμματα **Train loss, Test loss** άνα εποχή. 




## Sentence Polarity Dataset (MR)

<img src="./img/MR_50_loss.png?modified=12345678" width = "600" height = auto;>

## Semeval 2017 Task4-A (Semeval2017A)

<img src="./img/Semeval2017A_50_loss.png?modified=12345678" width = "600" height = auto;>


### Απαντήσεις Ερωτημάτων:

***Ερώτηση 1***: Γιατί αρχικοποιούμε το embedding layer με τα προ-εκπαιδευμένα word embeddings;

Για την αρχικοποίηση των βαρών του embedding layer είχαμε δύο επιλογές: να χρησιμοποιήσουμε βάρη απο pre-trained word embeddings ή να αρχικοποιήσουμε τα βάρη σε τυχαίες τιμές οι οποίες θα μαθαίνονταν κατά την εκπαίδευση του μοντέλου. Επιλέξαμε να χρησιμοποιήσουμε τα βάρη των pre-trained word embedding. 


Τα pre-trained embeddings αναπαριστούν τις λέξεις σε ένα διανυσματικό χώρο ώστε οι σημασιολογικά κοντινές λέξεις να έχουν μικρή απόσταση. Η εκπαίδευση τους έχει γίνει σε corpus εκατομμυρίων λέξεων, και αποτελούν ένα τεράστιο λεξικό αναπαραστάεων. 

Κείμενα με κοντινή αναπαράσταση στον embedded χώρο θα έχουν και κοντινό συναισθηματικό περιεχόμενο και άρα θα πρέπει να δώσουν κοινή έξοδο. Η πληροφορία αυτή ειναι ανεκτίμητη για έναν ταξινομητή συναισθήματος, και προφανώς δε μπορεί να συναχθεί απο τα δεδομένα εκπαίδευσης στον ίδιο βαθμό. Ακόμα οδηγούν σε καλύτερο generalization αφού λέξεις του test set μπορεί να διαθέτουν pre-trained embedding αλλα να μην γίνονται γνωστές απο το train set. 

Συμπερασματικά τα embeddings περιέχουν πλούσια σημασιολογική πληροφορία  και το DNN θα μπορέσει να αξιοποιήσει τις αναπαραστάσεις για να έχει καλύτερη απόδοση και να συγκλίνει πιο γρήγορα. 



***Ερώτηση 2***: Γιατί κρατάμε παγωμένα τα βάρη του embedding layer κατά την εκπαίδευση;

Οι λόγοι που κρατάμε παγωμένα τα βάρη του embedding layer κατά την εκπαίδευση είναι οι εξής:

* Τα βάρη που προκύπτουν από τα pre-trained word embeddings έχουν δημιουργηθεί από ένα μεγάλου μεγέθους corpus και δείχνουν μια συσχέτιση μεταξύ των λέξεων. Η συσχέτιση αυτή, μαθαίνεται μετά από την εκπαίδευση σε high-end μηχανές και χρησιμοποιώντας βέλτιστες παραμέτρους. Άρα, δεν χρειάζεται περαιτέρω ανανέωση των τιμών των βαρών με αποτέλεσμα να μειώνεται η υπολογιστή πολυπλοκότητα του αλγορίθμου.


* Αν συνεχίσουμε να ανανεώνουμε τις τιμές των βαρών των pre-trained word embeddings κατά την εκπαίδευση, ενδέχεται να αλλάξουμε τις συσχετίσεις που δείχνουν αρχκά τα embeddings και να μην έχουμε την επιθυμητή συμπεριφορά από το emdedding layer. Συγκεκριμένα, μια ανεπιθύμητη συμπεριφορά είναι το overfitting του DNN. Αν συνεχιστεί η ανανέωση των βαρών, το DNN θα εκπαιδευτεί πολύ καλά πάνω στο training set που έχουμε και έτσι δεν θα έχει καλή απόδοση σε κάποιο άλλο dataset. 

***Ερώτηση 3***: Γιατί βάζουμε μία μη γραμμική συνάρτηση ενεργοποίησης στο προτελευταίο layer; Τι διαφορά θα είχε αν είχαμε 2 ή περισσότερους γραμμικούς μετασχηματισμούς στη σειρά;


Οποιαδήποτε λειτουργία θέλουμε να κάνει το νευρωνικό δίκτυο που δημιουργούμε θέλουμε να την αναπαραστήσουμε σε μία υπολογιστική συνάρτηση.Για να πετύχουμε αυτό θα πρέπει να εφαρμόσουμε μια συνάρτηση ενεργοποίησης $f(x)$ έτσι ώστε το δίκτυο να γίνει πιο ισχυρό, να έχει την ικανότητα να μαθαίνει κάτι περίπλοκο και πολύπλοκο από τα δεδομένα και να αναπαριστά μη γραμμικές σύνθετες και αυθαίρετες αντιστοιχίες μεταξύ εισόδου και εξόδου. Ως εκ τούτου, χρησιμοποιώντας μη γραμμική ενεργοποίηση, είμαστε σε θέση να παράγουμε μη γραμμικές απεικονίσεις από τις εισόδους στις εξόδους αφού οι μη γραμμικές συναρτήσεις έχουν βαθμό μεγαλύτερο από ένα και έχουν καμπυλότητα όταν τις σχεδιάζουμε. Για προβλήματα classification πρέπει να μπορούμε να υπολογίσουμε μη γραμμικά decision boundaries.

Αν δεν είχαμε χρησιμοποιήσει μη-γραμμική συνάρτηση ενεργοποίησης, όσα layers και να βάζαμε στο DNN μας, θα συμπεριφερόταν σαν ένα single-layer perceptron αφού αν αθροίζαμε όλα τα layers του θα παίρναμε μια συνολική γραμμική συνάρτηση. Άρα, το μοντέλο θα προσπαθούσε κάθε φορά να αντιστοιχίσει την είσοδο με την έξοδο γραμμικά 
και έτσι θα είχαμε μία πολύ απλή αναπαράσταση των δεδομένων.  

***Ερώτηση 4***: Αν θεωρήσουμε ότι κάθε διάσταση του embedding χώρου αντιστοιχεί σε μία αφηρημένη έννοια, μπορείτε να δώσετε μία διαισθητική ερμηνεία για το τι περιγράφει η αναπαράσταση που φτιάξατε(κέντρο-βάρους).

Θεωρούμε έναν χώρο όπου κάθε διάσταση αντιστοιχίζεται και σε μία αφηρημένη έννοια. Τοτε για ένα διάνυσμα του χώρου $x = (x_1,x_2,...,x_n)$ η τιμή $x_i$ του διανύσματος εκφράζει την συσχέτιση με την έννοια της διάστασης $i$, πόσο κοντά δηλαδή σημασιολογικά είναι το κείμενο με την έννοια. Κάθε λέξη του κειμένου αποτελεί και ένα διάνυσμα που έχει μεγαλύτερη τιμή σε άλλες διάστασεις. Κάθε τιμή του μέσου όρου μας δίνει μια μετρική για το πόσο κοντά ειναι το κείμενο με την συγκεκριμένη έννοια. Για χαμηλές τιμές το κείμενο είναι ασυσχέτιστο με τις έννοιες και για υψηλές τιμές η συσχέτιση είναι μεγάλη. 


***Ερώτηση 5***: Αναφέρετε πιθανές αδυναμίες της συγκεκριμένης προσέγγισης για να αναπαραστήσουμε κείμενα.

* Στη συγκεκριμένη προσέγγιση δεν λαμβάνουμε υπόψιν την θέση της λέξης. Η σημασιολογία μίας λέξης αλλάζει ανάλογα με την θέση της στην πρόταση, και μια αρνητική λέξη μπορεί να μετατραπεί σε θετική σε διαφορετικά συμφραζόμενα. Ένας τρόπος να αντιμετοπιστεί αυτο το πρόβλημα είναι με εισαγωγή n-gram language models παράλληλα με τα embeddings. 


* Ακόμα, δεν λαμβάνουμε υπόψιν την σύνταξη της πρότασης, αφού αθροίζουμε απλά τις λέξεις χωρίς να ελέγχουμε για άλλα στοιχεία της πρότασης και τον  ρόλο λέξεων. 


* Τέλος, τα σημεία στίξης του κειμένου δεν επηρεάζουν την αναπαράσταση. Τα σημεία στίξης προσδίδουν διαφορετικό νόημα σε κάθε λέξη και δείχνουν διαφορετικό συναίσθημα για κάθε πρόταση. 

***Ερώτηση 6***: Τι συνέπειες έχουν τα μικρά και μεγάλα mini-batches στην εκπαίδευση των μοντέλων;

 

Για αρκετά μεγάλο batch size μπορούμε να πετύχουμε μια σταθερή εκτίμηση για το ποιο θα είναι το gradient για το συνολικό data set. Ακόμα ανεβαίνει σημαντικά η ταχύτητα εκπαίδευσης αφού το gradient υπολογίζεται αθροιστικά για ένα σύνολο δεδομένων και έπειτα πραγματοποιείται back propagation και gradient descend. Όσο πιο μικρό είναι το batch size τόσο λιγότερη ακριβής ειναι η εκτίμηση. Τα μικρά batches έχουν όμως θόρυβο αφού αποτελούνται από πολλά raw δεδομένα, το οποία μπορεί να βοηθήσει το μοντέλο να ξεφύγει απο τοπικά ελάχιστα και άρα να οδηγήσει σε καλύτερη βελτιστοποίηση. Όταν είναι πολύ μικρό όμως το μοντέλο παρουσιάζει έντονη ταλάντωση ή συγκλίνει πολυ αργά. Μικρά batch sizes μπορούν επίσης να μειώσουν το generalization error. 

Γενικά το σωστό μέγεθος batch είναι σημαντικό γιατί επηρεάζει την ταχύτητα σύγκλισης του μοντέλου, την υπολογιστικη πολυπλοκότητα αλλα και την τελική απόδοση. 

***Ερώτηση 7***: Συνήθως ανακατεύουμε την σειρά των mini-batches στα δεδομένα εκπαίδευσης σε κάθε εποχή. Μπορείτε να εξηγήσετε γιατί;


Θέλουμε να κάνουμε shuffle τη σειρά των mini-batches δεδομένα εκπαίδευσης για τους παρακάτω λόγους:

* Το DNN που έχουμε δημιουργήσει μπορεί εκτός από τις συναρτήσεις ενεργοποίησης που συνδέουν την είσοδο με την έξοδο να μάθει και την σειρά με την οποία δίνονται σε αυτό τα δεδομένα για την εκπαίδευση του. Άρα, αν κάνουμε shuffle ανά εποχή, το νευρωνικό βλέπει κάθε φορά τα δεδομένα με διαφορετική σειρά και δεν μπορεί να κάνει κάποια αντιστοίχιση της εισόδου με την έξοδο βασισμένο αυτή.


* Το shuffle των mini-batches ανά εποχή μπορεί επίσης να βοηθήσει το DNN να συγκλίνει πιο γρήγορα στο ολικό ελάχιστο κατά την εκτέλεση του stochastic gradient descent. Αυτό συμβαίνει γιατί αν σε μία εποχή ο αλγόριθμος έχει ‘’κολλήσει’’ σε τοπικό ελάχιστο, στην επόμενη εποχή έχει μεγάλη πιθανότητα να ‘’ξεκολλήσει’’ αφού τα δεδομένα θα δίνονται με διαφορετική σειρά στο νευρωνικό δίκτυο. 


* Αν κάνουμε shuffle τη σειρά εισάγουμε τυχαιότητα στη σειρά που δίνονται τα δεδομένα στο δίκτυο ανά εποχή και έτσι το loss που παίρνουμε σαν αποτέλεσμα είναι πιο αμερόληπτο.


***Ερώτηση 8:*** Αξιολόγηση του ζητούμενου 10

Παραπάνω έχουμε τυπώσει **classification report** που περιλαμβάνει F1_score (macro) και recall (macro). Ακόμα παραθέτουμε τις καμπύλες εκπαίδευσης. 

Παρατηρούμε ότι οι καμπύλες έιναι φυσιολογικές με το loss να πέφτει και στα δύο σύνολα. Η τιμή learning rate που επιλέξαμε είναι καλή αλλιώς θα βλέπαμε απότομη αύξηση του loss. 

Στο **MR** dataset παρατηρούμε μεγάλη ταλάντωση του loss function στο test set που σημαίνει οτι δεν γίνεται καλο generalization και απαιτεί περαιτέρω διερεύνηση.  

Στο **Semeval2017A** η καμπύλη εκπαίδευσης παραπέμπει σε καλά αποτελέσματα, με το loss να πέφτει σταθερά και στα δύο σύνολα. 

Γενικά δεν παρατηρείται overfitting όπως είναι αναμενόμενο λόγω της πολυπλοκότητας των δεδομένων και του προβλήματος, αλλα και τον μικρό αριθμό εποχών. Για μεγαλύτερο αριθμό εποχών αναμένουμε καλύτερα αποτελέσματα. 

Η ακρίβεια είναι **0.69** και **0.58** αντίστοιχα. Μεγαλύτερη στο **MR** όπως περιμένα αφού ειναι ευκολότερο πρόβλημα. Τα f1_score ως καλύτερες μετρικές απόδοσης ακολουθούν το accuracy. Αναφέρουμε οτι στο **Semeval2017A** η κλάση 1 (neutral) πάει σημαντικά καλύτερα απο τις άλλες (10% διαφορά στην ακρίβεια). Ακόμα η κλάση 0 (negative) έχει αρκετά χαμηλή τιμή recall **0.43**. Για το 
**MR** οι μετρικές είναι πολυ καλές.


In [29]:
!python main.py DNN_mean

loading word embeddings...
Loaded word embeddings from cache.

Question 1:


Labels for 10 first training examples:

Original: ['neutral' 'neutral' 'negative' 'neutral' 'positive' 'negative' 'neutral'
 'neutral' 'neutral' 'neutral']

After LabelEncoder: [1 1 0 1 2 0 1 1 1 1]

the

Question 2:

Tokenized sample:
['@', 'seemonterey', 'lost', '-', 'sony', 'cell', 'phone', 'with', 'holiday', 'photos', '.', 'early', 'fri', 'morning', ',', 'montreal', 'transit', 'plaza', 'or', 'no', '.', '13', 'bus', 'to', 'airport', '.', 'reward', '!', 'plz', 'rt', '.']

Tokenized sample:
['@', 'personasoda', 'well', 'yeah', ',', 'that', "'s", 'third', 'parties', '.', 'sony', 'itself', 'is', "n't", 'putting', 'out', 'actual', 'games', 'for', 'it', '.', 'it', "'s", 'got', '1-2', 'yrs', 'of', '3rd', 'party', 'support', 'left', '.']

Tokenized sample:
['sony', 'rewards', 'app', 'is', 'like', 'a', 'lot', 'of', '19', 'y.o', 'female', 'singers', 'and', 'a', 'non', 'retro', 'sale', '.', '2nd', 'one', 'with', 'no',

 [========================================] ...Epoch 16, Loss: 1.0224
 Epoch 16, Train loss: 0.8189
           Test loss: 0.8448
 [========================================] ...Epoch 17, Loss: 0.8647
 Epoch 17, Train loss: 0.8195
           Test loss: 0.8541
 [========================================] ...Epoch 18, Loss: 0.8202
 Epoch 18, Train loss: 0.8157
           Test loss: 0.8519
 [========================================] ...Epoch 19, Loss: 1.0188
 Epoch 19, Train loss: 0.8190
           Test loss: 0.8428
 [========================================] ...Epoch 20, Loss: 0.8780
 Epoch 20, Train loss: 0.8155
           Test loss: 0.8481
 [========================================] ...Epoch 21, Loss: 0.8144
 Epoch 21, Train loss: 0.8422
           Test loss: 0.9086
 [========================================] ...Epoch 22, Loss: 0.8970
 Epoch 22, Train loss: 0.8173
           Test loss: 0.8487
 [========================================] ...Epoch 23, Loss: 0.8410
 Epoch 23, Train loss: 0.81

In [34]:

from models import BaselineDNN,BaseLSTM

model = BaselineDNN(output_size=n_classes,  # EX8
                    embeddings=embeddings,
                    method = 'mean',
                    attention_size=60,
                    trainable_emb=EMB_TRAINABLE,tf_idf = TF_IDF)

import torch
model_name = 'DNN_mean'
from training import train_dataset, eval_dataset
from sklearn.metrics import classification_report


NameError: name 'model' is not defined

In [7]:
!python main.py DNN_pooling

loading word embeddings...
Loaded word embeddings from cache.

Question 1:


Labels for 10 first training examples:

Original: ['neutral' 'neutral' 'negative' 'neutral' 'positive' 'negative' 'neutral'
 'neutral' 'neutral' 'neutral']

After LabelEncoder: [1 1 0 1 2 0 1 1 1 1]

the

Question 2:

Tokenized sample:
['@', 'seemonterey', 'lost', '-', 'sony', 'cell', 'phone', 'with', 'holiday', 'photos', '.', 'early', 'fri', 'morning', ',', 'montreal', 'transit', 'plaza', 'or', 'no', '.', '13', 'bus', 'to', 'airport', '.', 'reward', '!', 'plz', 'rt', '.']

Tokenized sample:
['@', 'personasoda', 'well', 'yeah', ',', 'that', "'s", 'third', 'parties', '.', 'sony', 'itself', 'is', "n't", 'putting', 'out', 'actual', 'games', 'for', 'it', '.', 'it', "'s", 'got', '1-2', 'yrs', 'of', '3rd', 'party', 'support', 'left', '.']

Tokenized sample:
['sony', 'rewards', 'app', 'is', 'like', 'a', 'lot', 'of', '19', 'y.o', 'female', 'singers', 'and', 'a', 'non', 'retro', 'sale', '.', '2nd', 'one', 'with', 'no',

In [8]:
!python main.py DNN_attention

loading word embeddings...
Loaded word embeddings from cache.

Question 1:


Labels for 10 first training examples:

Original: ['neutral' 'neutral' 'negative' 'neutral' 'positive' 'negative' 'neutral'
 'neutral' 'neutral' 'neutral']

After LabelEncoder: [1 1 0 1 2 0 1 1 1 1]



In [9]:
!python main.py LSTM_mean

^C
Traceback (most recent call last):
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/numpy/core/__init__.py", line 24, in <module>
    from . import multiarray
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/numpy/core/multiarray.py", line 14, in <module>
    from . import overrides
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/numpy/core/overrides.py", line 7, in <module>
    from numpy.core._multiarray_umath import (
ImportError: PyCapsule_Import could not import module "datetime"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "main.py", line 6, in <module>
    from sklearn.exceptions import UndefinedMetricWarning
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/sklearn/__init__.py", line 82, in <module>
    from .base import clone
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/sklearn/base.py", line 17, in <module>
    import numpy as np
  File "/home/nikit

In [10]:
!python main.py LSTM_pooling

loading word embeddings...
Loaded word embeddings from cache.

Question 1:


Labels for 10 first training examples:

Original: ['neutral' 'neutral' 'negative' 'neutral' 'positive' 'negative' 'neutral'
 'neutral' 'neutral' 'neutral']

After LabelEncoder: [1 1 0 1 2 0 1 1 1 1]



In [11]:
!python main.py LSTM_attention

^C
Traceback (most recent call last):
  File "main.py", line 6, in <module>
    from sklearn.exceptions import UndefinedMetricWarning
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/sklearn/__init__.py", line 82, in <module>
    from .base import clone
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/sklearn/base.py", line 17, in <module>
    import numpy as np
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/numpy/__init__.py", line 152, in <module>
    from . import random
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/numpy/random/__init__.py", line 181, in <module>
    from . import _pickle
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/numpy/random/_pickle.py", line 1, in <module>
    from .mtrand import RandomState
  File "_bit_generator.pxd", line 14, in init numpy.random.mtrand
  File "_bit_generator.pyx", line 40, in init numpy.random._bit_generator
  File "/home/nikitas/anaconda3/lib/python3.7/secrets.py", line 15, i

In [12]:
!python main.py LSTM_pooling_B


^C
Traceback (most recent call last):
  File "main.py", line 6, in <module>
    from sklearn.exceptions import UndefinedMetricWarning
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/sklearn/__init__.py", line 82, in <module>
    from .base import clone
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/sklearn/base.py", line 20, in <module>
    from .utils import _IS_32BIT
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py", line 27, in <module>
    from .fixes import np_version
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/sklearn/utils/fixes.py", line 18, in <module>
    import scipy.stats
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/scipy/stats/__init__.py", line 384, in <module>
    from .stats import *
  File "/home/nikitas/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py", line 180, in <module>
    from scipy.ndimage import measurements
  File "/home/nikitas/anaconda3/lib/python3.7/site

In [13]:
!python main.py LSTM_attention_B

loading word embeddings...
